In [ ]:
!rm tallies.xml tallies.out materials.xml geometry.xml settings.xml


import openmc
import openmc.deplete
from pathlib import Path
import math
import openmc_depletion_plotter
import re

v = openmc.Material()
v.add_element('V', 1, percent_type='ao')
v.set_density('g/cm3', 6.1)

ring_thickness = 1 #cm
inner_rad = 114 #cm
outer_rad = inner_rad + ring_thickness
#Volume calc assuming ring as tall as thick
ring_vol = ring_thickness * math.pi * (outer_rad**2 - inner_rad ** 2)
v.volume = ring_vol
v.depletable = True
materials = openmc.Materials([v])
materials.export_to_xml()


# GEOMETRY
outer_sphere = openmc.Sphere(r=outer_rad, boundary_type='vacuum')
inner_cyl = openmc.ZCylinder(r=inner_rad)
outer_cyl = openmc.ZCylinder(r=outer_rad, boundary_type='reflective')
bottom_plane = openmc.ZPlane(z0=0)
top_plane = openmc.ZPlane(z0=1)

ring_region = +inner_cyl & -outer_cyl & +bottom_plane & -top_plane
ring_cell = openmc.Cell(region=ring_region)
ring_cell.fill = v
void_region = -outer_sphere & ~ring_region
void_cell = openmc.Cell(region=void_region)
void_cell.fill = None
geometry = openmc.Geometry([ring_cell, void_cell])


# SOURCE
source = openmc.IndependentSource()
source.space = openmc.stats.Point((0,0,0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14.1e6], [1])
source.particles = 'neutron'

# SETTINGS

settings = openmc.Settings(
    batches = 2,
    inactive = 0,
    particles = 10000,
    source = source,
    run_mode = 'fixed source'
)


timesteps_and_source_rates = [
    (365*24*60*60, 1e20)
]
for i in range(12):
    timesteps_and_source_rates.append((30*24*60*60, 0))
for i in range(9):
    timesteps_and_source_rates.append((365*24*60*60, 0))
for i in range(9):
    timesteps_and_source_rates.append((10*365*24*60*60, 0))
for i in range(9):
    timesteps_and_source_rates.append((100*365*24*60*60, 0))
    
    
timesteps = [item[0] for item in timesteps_and_source_rates]
source_rates = [item[1] for item in timesteps_and_source_rates]

# VIZ
color_assignment = {ring_cell: 'blue', void_cell: 'red'}

plot = geometry.plot(basis='xz', color_by='cell', colors=color_assignment)
plot.figure.savefig('xz-cell.png')

plot = geometry.plot(basis='xy', color_by='cell',  colors=color_assignment)
plot.figure.savefig('xy-cell.png')

plot = geometry.plot(basis='yz', color_by='cell',  colors=color_assignment)
plot.figure.savefig('yz-cell.png')

#Tallies

chain = openmc.deplete.Chain.from_xml(openmc.config['chain_file'])
nuclides = [nuclide.name for nuclide in chain.nuclides]


energy_filter = openmc.EnergyFilter.from_group_structure('CCFE-709')
ccfe_tally = openmc.Tally(name="ccfe_tally")
ccfe_tally.filters.append(energy_filter)
ccfe_tally.scores.append('flux')
tallies = openmc.Tallies([ccfe_tally])

dose_tally = openmc.Tally(name="dose_rate")
dose_tally.filters = [openmc.CellFilter(ring_cell)]
dose_tally.scores = ['flux']
dose_tally.nuclides = ['total']
tallies.append(dose_tally)

heat_tally = openmc.Tally(name='heat')
heat_tally.filters = [openmc.CellFilter(ring_cell)]
heat_tally.scores = ['heating']
tallies.append(heat_tally)

damage_tally = openmc.Tally(name='damage_energy')
damage_tally.filters = [openmc.CellFilter(ring_cell)]
damage_tally.scores = ['damage-energy']
tallies.append(damage_tally)

tallies.export_to_xml()

#Deplete

model = openmc.model.Model(geometry, materials, settings, tallies)

# Deplete

model.deplete(
    timesteps,
    source_rates=source_rates,
    method = "predictor",
    operator_kwargs={
        "normalization_mode": "source-rate",
        "chain_file": openmc.config['chain_file'],
        "reduce_chain_level": 5,
        "reduce_chain": True
    }
)
    
def convert_time_units(time):
    if time > 24 * 365:
        return time / (24 * 365), 'y'
    elif time > 24:
        return time / 24, 'd'
    else:
        return time, 'h'
    
def get_long_lived_limit(nuc):
    match nuc:
        case 'C14': 
            return 80 #assumes C14 in activated metal
        case 'Ni59': 
            return 220
        case 'Nb94':
            return .2
        case 'Tc99':
            return 3
        case 'I129':
            return .08
        case 'Pu241':
            return 3500
        case 'Cm242':
            return 20000
        #alpha-emitting transuranic elements with half life > 5 year
        case 'Np237' | 'Pu238' | 'Pu239' | 'Pu240' | 'Pu242' | 'Pu244' | 'Am241' | 'Am243' | 'Cm243' | 'Cm244' | 'Cm245' | 'Cm246' | 'Cm247' | 'Cm248' | 'Cm250' | 'Bk247' | 'Cf249' | 'Cf250' | 'Cf251': 
            return 100
        case _:
            return 0

def get_short_lived_limit(nuc):
    match nuc:
        
    
results = openmc.deplete.Results("depletion_results.h5")
prev_time = 0
total_mass = v.volume * v.density
print("total mass: ", total_mass)
with open('v.out', 'w') as file:
    for i, time in enumerate(results.get_times(time_units = "h")):
        d_time = time - prev_time
        prev_time = time
        converted_d_time, converted_d_time_unit = convert_time_units(d_time)
        converted_time, converted_time_unit = convert_time_units(time)
        print(f"\nDepletion step {i}. Time interval: {converted_d_time} {converted_d_time_unit}. Total time elapsed: {converted_time} {converted_time_unit}.", file = file)
        print("Nuclide   Atoms         Mass (grams)     Activity (Bq)  g-Energy (eV)  half life", file = file)
        long_lived_concentration = 0;
        short_lived_concentration = 0;
        for nuc in nuclides:
            try:
                 # Get the number of atoms for the nuclide at this depletion step
                _, num_atoms = results.get_atoms(v, nuc)
                if num_atoms[i] > 0:
                    mass_amus = openmc.data.atomic_mass(nuc) * num_atoms[i]
                    mass_grams = mass_amus / 6.022e23
                    activity = openmc.data.decay_constant(nuc) * num_atoms[i]
                    if openmc.data.decay_photon_energy(nuc) is not None:
                        g_energy_eV = openmc.data.decay_photon_energy(nuc).integral() * num_atoms[i]
                    else:
                        g_energy_eV = 0
                    h_life = openmc.data.half_life(nuc)
                    h_life_text = "Stable"
                    if h_life != None:
                        h_life_num, h_life_unit = convert_time_units(h_life)
                        h_life_text = str(f"{h_life_num:3g}{h_life_unit}")
                    space1 = " " * (10-len(nuc))
                    space2 = " " * (14-len(str(f"{num_atoms[i]:5g}")))
                    space3 = " " * (17-len(str(f"{mass_grams:3g}")))
                    space4 = " " * (15-len(str(f"{activity:3g}")))
                    space5 = " " * (15-len(str(f"{g_energy_eV:3g}")))
                    print(f"{nuc}{space1}{num_atoms[i]:5g}{space2}{mass_grams:3g}{space3}{activity:3g}{space4}{g_energy_eV:3g}{space5}{h_life_text}", file = file)
                    percentage = (openmc.data.atomic_mass(nuc) * num_atoms[i] / 6.022e23) / total_mass
                    activity_bq_per_cm_3 = openmc.data.decay_constant(nuc) * num_atoms[i] / v.volume
                    activity_cu_per_m_3 = (activity_bq_per_cm_3 / 3.7e10) * 1e6
                    limit = get_long_lived_limit(nuc)
                    if limit > 0:
                        long_lived_contribution = percentage * activity_cu_per_m_3 / limit
                        if long_lived_contribution > 0:
                            long_lived_concentration += long_lived_contribution
                            print(f"{nuc}  percentage: {percentage} activity cu/m3: {activity_cu_per_m_3}  limit: {limit}  long lived contribution: {long_lived_contribution}", file=file)
                            print(f"long lived concentration: {long_lived_concentration}", file=file)

            except Exception as e:
                pass
        print(f"Long lived concentration: {long_lived_concentration}")
            
with open("fluxes", 'w') as file:
    sp = openmc.StatePoint('statepoint.2.h5')
    flux_values = sp.get_tally(name='ccfe_tally').get_values(scores=['flux'])
    sp.close()
    for flux in flux_values:
        print(f"{flux[0].item():.18e}", file = file)